In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
import pickle
import colorama

2022-10-19 10:33:37.543164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-19 10:33:37.543203: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('intents.json') as file:
  data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Pẹlẹ o ',
    ' Bawo ni o se wa?',
    ' Pẹlẹ o bawo ni ',
    ' Bawo ni nibe yen o ',
    ' kilode '],
   'responses': ['Pẹlẹ o',
    'Hello',
    ' Bawo ni o ṣe n ṣe',
    ' ikini fun ọ ',
    ' mo wa dada',
    'mo wa dada',
    'o dara o ṣeun']},
  {'tag': 'goodbye',
   'patterns': [' o dabọ ', ' ri ọ ', ' titi a o tun pade ', ' nigbamii '],
   'responses': ['O dara lati ba ọ sọrọ ', ' Ma a ri e laipe ', ' Sọ laipẹ!']},
  {'tag': 'command',
   'patterns': ['tan ina',
    'pa ina',
    'si ilekun',
    'tan fan',
    ' pa ilẹkun ',
    'Pa fan'],
   'responses': ['ina ti tan ',
    'mo ti pa ina naa',
    'mo ti silekun ',
    ' ololufẹ naa wa ni titan ',
    'ilekun ti wa ni pipade',
    'mo to pa fan']}]}

In [4]:
train_sentences = []
train_labels = []
labels = []
responses = []

In [5]:
for intent in data['intents']:
  for pattern in intent["patterns"]:
    train_sentences.append(pattern)
    train_labels.append(intent["tag"])
  responses.append(intent["responses"])

  if (intent["tag"] not in labels):
    labels.append(intent['tag'])

In [6]:
train_labels, train_sentences, labels, responses

(['greeting',
  'greeting',
  'greeting',
  'greeting',
  'greeting',
  'goodbye',
  'goodbye',
  'goodbye',
  'goodbye',
  'command',
  'command',
  'command',
  'command',
  'command',
  'command'],
 ['Pẹlẹ o ',
  ' Bawo ni o se wa?',
  ' Pẹlẹ o bawo ni ',
  ' Bawo ni nibe yen o ',
  ' kilode ',
  ' o dabọ ',
  ' ri ọ ',
  ' titi a o tun pade ',
  ' nigbamii ',
  'tan ina',
  'pa ina',
  'si ilekun',
  'tan fan',
  ' pa ilẹkun ',
  'Pa fan'],
 ['greeting', 'goodbye', 'command'],
 [['Pẹlẹ o',
   'Hello',
   ' Bawo ni o ṣe n ṣe',
   ' ikini fun ọ ',
   ' mo wa dada',
   'mo wa dada',
   'o dara o ṣeun'],
  ['O dara lati ba ọ sọrọ ', ' Ma a ri e laipe ', ' Sọ laipẹ!'],
  ['ina ti tan ',
   'mo ti pa ina naa',
   'mo ti silekun ',
   ' ololufẹ naa wa ni titan ',
   'ilekun ti wa ni pipade',
   'mo to pa fan']])

In [7]:
num_classes = len(labels)
num_classes

3

In [8]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_labels)

In [9]:
train_labels

array([2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])

In [10]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [11]:
tokenizer = Tokenizer(
  num_words=vocab_size, oov_token=oov_token
)

In [12]:
tokenizer.fit_on_texts(train_sentences)

In [13]:
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'o': 2,
 'bawo': 3,
 'ni': 4,
 'pa': 5,
 'pẹlẹ': 6,
 'tan': 7,
 'ina': 8,
 'fan': 9,
 'se': 10,
 'wa': 11,
 'nibe': 12,
 'yen': 13,
 'kilode': 14,
 'dabọ': 15,
 'ri': 16,
 'ọ': 17,
 'titi': 18,
 'a': 19,
 'tun': 20,
 'pade': 21,
 'nigbamii': 22,
 'si': 23,
 'ilekun': 24,
 'ilẹkun': 25}

In [15]:
sequences = tokenizer.texts_to_sequences(train_sentences)
sequences

[[6, 2],
 [3, 4, 2, 10, 11],
 [6, 2, 3, 4],
 [3, 4, 12, 13, 2],
 [14],
 [2, 15],
 [16, 17],
 [18, 19, 2, 20, 21],
 [22],
 [7, 8],
 [5, 8],
 [23, 24],
 [7, 9],
 [5, 25],
 [5, 9]]

In [16]:
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  6,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,
         4,  2, 10, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         6,  2,  3,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,
         4, 12, 13,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 14],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  2, 15],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 16, 17],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18,
        19,  2, 20, 21],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 22],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  7,  8],
       [ 0,  0,  0,  0,  0,  0

In [17]:
model = Sequential([
  Embedding(vocab_size, embedding_dim, input_length=max_len),
  GlobalAveragePooling1D(),
  Dense(16, activation="relu"),
  Dense(16, activation="relu"),
  Dense(num_classes, activation="softmax")
])

2022-10-19 10:36:40.746571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-19 10:36:40.763193: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-19 10:36:40.763275: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xviix): /proc/driver/nvidia/version does not exist
2022-10-19 10:36:40.791066: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(
  loss="sparse_categorical_crossentropy",
  optimizer='adam',
  metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 3)                 51        
                                                                 
Total params: 16,595
Trainable params: 16,595
Non-trainable params: 0
____________________________________________________

In [20]:
history = model.fit(padded_sequences, np.array(train_labels), epochs=500)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 1.0995 - accuracy: 0.2667
Epoch 2/500
1/1 [==============================] - 0s 6ms/step - loss: 1.0990 - accuracy: 0.2667
Epoch 3/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0984 - accuracy: 0.2667
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0981 - accuracy: 0.4000
Epoch 5/500
1/1 [==============================] - 0s 10ms/step - loss: 1.0977 - accuracy: 0.4000
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0973 - accuracy: 0.4000
Epoch 7/500
1/1 [==============================] - 0s 4ms/step - loss: 1.0969 - accuracy: 0.4000
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 1.0965 - accuracy: 0.4000
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 1.0961 - accuracy: 0.4000
Epoch 10/500
1/1 [==============================] - 0s 13ms/step - loss: 1.0957 - accuracy: 0.4000
Epoch 11/500
1/1 [==========

In [21]:
model.save("chat_model")

2022-10-19 10:37:21.242073: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [22]:
with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as enc:
  pickle.dump(encoder, enc, protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
colorama.init()

In [23]:
from colorama import Fore, Style, Back

In [26]:
def inference(input):

  # load model
  model = keras.models.load_model('chat_model')

  # load tokenizer
  with open('tokenizer.pickle', 'rb') as tok:
    tokenizer = pickle.load(tok)

  # load label encoder
  with open('label_encoder.pickle', 'rb') as enc:
    encoder = pickle.load(enc)

  sequences = tokenizer.texts_to_sequences([input])
  padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

  prediction = model.predict(padded_sequences)

  pred_tag = encoder.inverse_transform([np.argmax(prediction)])

  return (pred_tag)

In [27]:
inference("pa fan")

array(['command'], dtype='<U8')

array(['greeting'], dtype='<U8')